# CHEM 1000 - Spring 2024
Prof. Geoffrey Hutchison, University of Pittsburgh

## Recitation - Curve Fitting

For this recitation, we'll focus on:
- Use of `scipy` `curve_fit` for general curve fitting - not just standard linear or quadratic fits
- Determining the standard error of parameters
- Evaluating the "goodness of fit" using both $R^2$ and errors (RMSE or MAE)
---

**Attribution**

Much of this material has been adapted from [Using scipy for data fitting](https://education.molssi.org/python-data-analysis/03-data-fitting/index.html) by Prof. Ashley Ringer McDonald and [Python Data and Scripting for Biochemists](http://education.molssi.org/python-scripting-biochemistry/chapters/nonlinear_regression_part_2.html) by Dr. Jessica Nash.

## Fitting Nonlinear Fuctions

Earlier, we used the `linregress()` function from the `scipy.stats` library to perform linear regression. This is sufficient for a standard linear fit to the data. 

Python contains tools that also enable us to fit data to an equation that we have defined, with two or more paramaters. So it is ideal for nonlinear curve fitting. You can also calculate the standard error for any parameter in a functional fit.

The basic steps are:

1. Create a list or numpy array of your independent variable (your x values). You might read this data in from another source, like a CSV file.
2. Create a list of numpy array of your depedent variables (your y values). You might read this data in from another source, like a CSV file.
3. Create a function for the equation you want to fit. The function should accept as inputs the independent variable(s) and all the parameters to be fit.
4. Import and use the function `curve_fit` to fit your data.
5. Extract the fit parameters from the output of curve_fit.
6. Use your function to calculate y values using your fit model to see how well your model fits the data.
7. Graph your original data and the fit equation.

We'll start with two simple cases, and then try some chemical kinetic data.

## What Kind of Peak?

In [ ]:
# First, we'll import some modules including scipy.optimize.curve_fit
import numpy as np
from scipy.optimize import curve_fit

Now we will consider a set of x,y-data. This data has one independent variable (our x values) and one dependent variable (our y values). We will recast the data as numpy arrays, so we can use numpy features when we are evaluating our data. It is often very helpful to look at a plot of the data when deciding what functional form to fit.

In [ ]:
xdata = [ -10.0, -9.0, -8.0, -7.0, -6.0, -5.0, -4.0, -3.0, -2.0, -1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
ydata = [1.2, 4.2, 6.7, 8.3, 10.6, 11.7, 13.5, 14.5, 15.7, 16.1, 16.6, 16.0, 15.4, 14.4, 14.2, 12.7, 10.3, 8.6, 6.1, 3.9, 2.1]

#Recast xdata and ydata into numpy arrays so we can use their handy features
xdata = np.asarray(xdata)
ydata = np.asarray(ydata)

It's probably helpful to plot it, so we'll import matplotlib.

In [ ]:
# import numpy and matplotlib
import matplotlib.pyplot as plt
# insert plots in the notebook
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# bump up the font sizes
plt.style.use('./chem1000.mplstyle')

In [ ]:
plt.plot(xdata, ydata, 'o')

Okay, it's definitely not linear. Looks maybe like a Gaussian peak?

This data could probably be fit to many functional forms. We will try three different functional forms. (Looking at data and knowing what function it might fit is non-trivial and beyond the scope of this lesson. For purposes of this lesson, we will simply fit the data to given functional forms.)

Note that this particular data is centered around $x = 0$ so we will simplify the functional forms:

- Gaussian $y=A e^{-B x^{2}}$
- Lorentzian $y = \frac{1}{\pi C}\left[\frac{C^{2}}{x^{2}+C^{2}}\right]$
- Cosine $y = D\cos(Ex)$

Note that many chemical spectra have either Gaussian (UV) or Lorentzian (NMR) peaks.

## Part A. Is it Gaussian?

First, let’s fit the data to the Gaussian function. Our goal is to find the values of A and B that best fit our data. First, we need to write a python function for the Gaussian function equation. The function should accept as inputs the independent varible (the x-values) and all the parameters that will be fit.

In [ ]:
# define the Gaussian function with parameters A and B
def Gauss(x, A, B):
    return A*np.exp(-1*B*x**2)

We will use the function `curve_fit` from the python module `scipy.optimize` to fit our data. It uses non-linear least squares to fit data to a functional form. You can learn more about `curve_fit` by using the `help` function within the Jupyter notebook or from the [scipy online documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html).

The `curve_fit` function has three required inputs:
* the function you want to fit
* the x-data
* the y-data you are fitting.

There are two outputs:
* The first is an array of the optimal values of the parameters. 
* The second a matrix of the estimated covariance of the parameters from which you can calculate the standard error for the parameters.


In [ ]:
parameters, covariance = curve_fit(Gauss, xdata, ydata)

The optimized values of A and B are now stored in the list `parameters`. From this, we can extract our best fit values of A and B and print them.

In [ ]:
fit_A = parameters[0]
fit_B = parameters[1]
print(fit_A)
print(fit_B)

Now we want to see how well our fit equation matched our data. To do this, we will calculate values of y, using our function and the fit values of A and B, and then we will make a plot to compare those calculated values to our data.

In [ ]:
fit_y = Gauss(xdata, fit_A, fit_B)
plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()

To calculate the standard error of the parameters from the covariance, you take the square root of the diagonal elements of the matrix. You can do this in one line using functions from `numpy`.

In [ ]:
SE = np.sqrt(np.diag(covariance))
SE_A = SE[0]
SE_B = SE[1]

print(F'The value of A is {fit_A:.5f} with standard error of {SE_A:.5f}.')
print(F'The value of B is {fit_B:.5f} with standard error of {SE_B:.5f}.')

You might be curious about the $R^2$. To do that from `curve_fit` we have to calculate it manually. 

It's also worthwhile to calculate the RMSE = root mean squared error or MAE = mean absolute error. These tell us roughly how close the points are to the fit line.

In [ ]:
# Calculate the R^2

residuals = ydata - Gauss(xdata, fit_A, fit_B)
ss_res = np.sum(residuals**2) # sum of squares of the residuals
ss_tot = np.sum((ydata-np.mean(ydata))**2) # total sum of squares
r_squared = 1 - (ss_res / ss_tot)
print('R-squared is: ', r_squared)

# Calculate the mean absolute error
mae = np.mean(np.abs(residuals))
print('MAE is: ', mae)

## Part B: Is it Lorentzian?

Now we'll try the same thing but with a Lorentzian fit.

In [ ]:
# define the Lorentzian function with parameter C
def Lorentzian(x, C):
    return (C**2 / (x**2 + C**2)/(np.pi*C))

parameters, covariance = curve_fit(Lorentzian, xdata, ydata)

fit_C = parameters[0]

fit_y = Lorentzian(xdata, fit_C)
plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_y, '-', label='fit')
plt.legend()

Okay. Maybe not.

## Part C: Is it Cosine?

Last choice, let's try a scaled cosine:

In [ ]:
def cos_func(x, D, E):
    y = D*np.cos(E*x)
    return y

parameters, covariance = curve_fit(cos_func, xdata, ydata)
fit_D = parameters[0]
fit_E = parameters[1]

fit_cosine = cos_func(xdata, fit_D, fit_E)

plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_cosine, '-', label='fit')

To fix this, we need to give a guess for what we think our parameters are. Thinking about the form of the cosine function, the height of the function is controlled by the D parameter. Looking at our graph, it seems the value of D is somewhere between 15 and 17, so we will guess 16. Similarly, the E parameter tells us how many cycles occur over the 0 to 2 $\pi$ interval. This is a very wide graph; there is clearly much less than one cycle between 0 and 2 $\pi$, so we will guess that E is 0.1. To incorporate these guesses into our code, we will create a new array called guess. We then specify our guess when we call curve_fit.

In [ ]:
guess = [16, 0.1]
parameters, covariance = curve_fit(cos_func, xdata, ydata, p0=guess)
fit_D = parameters[0]
fit_E = parameters[1]

fit_cosine = cos_func(xdata, fit_D, fit_E)

plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_cosine, '-', label='fit')

In [ ]:
SE = np.sqrt(np.diag(covariance))
SE_D = SE[0]
SE_E = SE[1]

print(F'The value of D is {fit_D:.5f} with standard error of {SE_D:.5f}.')
print(F'The value of E is {fit_E:.5f} with standard error of {SE_E:.5f}.')

In [ ]:
# Calculate the R^2

residuals = ydata - cos_func(xdata, fit_D, fit_E)
ss_res = np.sum(residuals**2) # sum of squares of the residuals
ss_tot = np.sum((ydata-np.mean(ydata))**2) # total sum of squares
r_squared = 1 - (ss_res / ss_tot)
print('R-squared is: ', r_squared)

# Calculate the mean absolute error
mae = np.mean(np.abs(residuals))
print('MAE is: ', mae)

Okay, check back to the Gaussian fit.. which one has the better $R^2$ and MAE?

## Part D: Michaelis Menten

The [Michaelis-Menten equation](https://en.wikipedia.org/wiki/Michaelis–Menten_kinetics) describes kinetics for enzyme-catalyzed reactions. It involves an enzyme, E, binding to a substrate, S, to form a complex, ES, which in turn releases a product, P, regenerating the original enzyme:

$$
\mathrm{E}+\mathrm{S} \underset{k_{r}}{\stackrel{k_{f}}{\rightleftharpoons}} \mathrm{ES} \stackrel{k_{\mathrm{cat}}}{\longrightarrow} \mathrm{E}+\mathrm{P}
$$

Simplified, the reaction rate $v$ is given by:

$$
v =\frac{V_{\max } *[S]}{K_{m}+[S]}
$$

where $V_{\max}$ is the maximum rate achieved by the system (at long time), $K_m$ is the Michaelis constant, and $[S]$ is the concentration of substrate.

Mainly, we're going to fit some data to see if it matches Michaelis-Menten kinetics.

In [ ]:
# Set up a pandas dataframe from the MM_data.csv file we grabbed
# import the data
import pandas as pd
rates_df = pd.read_csv('MM_data.csv')

# inspect the data to make sure you have substrate concentrations and initial velocities
rates_df.head()

In [ ]:
# Create your independent and dependent variables for curve fitting.
ydata = rates_df['slopes']
xdata = rates_df['pNPP (mM)']

In [ ]:
# Inspect your data
plt.plot(xdata, ydata, 'o', label='data')

Okay, now let's try curve fitting!

In [ ]:
def MM(SubConc, Vmax, Km):
    return Vmax * SubConc/(Km + SubConc)

In [ ]:
parameters, covariance = curve_fit(MM, xdata, ydata)
fit_Vmax = parameters[0]
fit_Km = parameters[1]

fit_MM = MM(xdata, fit_Vmax, fit_Km)

plt.plot(xdata, ydata, 'o', label='data')
plt.plot(xdata, fit_MM, '-', label='fit')

In [ ]:
SE = np.sqrt(np.diag(covariance))
SE_Vmax = SE[0]
SE_Km = SE[1]

print(F'The value of Vmax is {fit_Vmax:.5f} +/- {SE_Vmax:.5f}.')
print(F'The value of E is {fit_Km:.5f} +/- {SE_Km:.5f}.')

In [ ]:
# Calculate the R^2

residuals = ydata - MM(xdata, fit_Vmax, fit_Km)
ss_res = np.sum(residuals**2) # sum of squares of the residuals
ss_tot = np.sum((ydata-np.mean(ydata))**2) # total sum of squares
r_squared = 1 - (ss_res / ss_tot)
print('R-squared is: ', r_squared)

# Calculate the mean absolute error
mae = np.mean(np.abs(residuals))
print('MAE is: ', mae)

-------
This notebook is from Prof. Geoffrey Hutchison, University of Pittsburgh
https://github.com/ghutchis/chem1000

Much of this material has been adapted from [Using scipy for data fitting](https://education.molssi.org/python-data-analysis/03-data-fitting/index.html) by Prof. Ashley Ringer McDonald and [Python Data and Scripting for Biochemists](http://education.molssi.org/python-scripting-biochemistry/chapters/nonlinear_regression_part_2.html) by Dr. Jessica Nash.

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>